# QLSAs Workflow

Use this notebook as a quick reference for configuring and running a solver.

## 1. Choose an algorithm
- `HHL`, `VQLSA`, `QHD`, or another solver in `QLSAs`
- Set the algorithm-specific hyperparameters.  For example:
  - `HHL`: number of QPE qubits, precision targets
  - `VQLSA`: optimizer, `maxiter`, initial state ansatz
  - Any solver: shot count (often scale with problem size)

## 2. Decide on iterative refinement
- Should we enable IR?
- Configure stopping criteria:
  - Maximum iterations
  - Residual tolerance or improvement threshold

## 3. Select a backend
- Vendor: IBM, Quantinuum, etc.
- Execution target:
  - Real hardware
  - Simulator (noiseless or noisy model)
- For Qiskit backends, choose the instance or fake backend

Document the chosen settings in the following cells before running experiments.

## Step 1: Setup

In [ ]:
# imports
import qiskit
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
from QLSAs.linear_systems_problems.random_matrix_generator import generate_problem

problem_size = 4  
condition_number = 10.0  
sparsity = 0.5

problem = generate_problem(
    n=problem_size,
    cond_number=condition_number,
    sparsity=sparsity,
)

A = problem["A"]
b = problem["b"]

## Step 2: Configuration

In [ ]:
from QLSAs.algorithms import HHL, VQLSA, QHD, NumpyLinearSolver
from QSLAs import Solver, Refiner, BackendConfig

hhl = HHL(
    n_qpe_qubits=3,
    t0=2 * np.pi,
)

optimizer = None  # supply a qiskit optimizer instance
initial_state = None  # supply an optional ansatz circuit
vqlsa = VQLSA(
    optimizer=optimizer,
    maxiter=200,
    initial_state=initial_state,
)

qhd = QHD()

numpy_solver = NumpyLinearSolver()

In [ ]:
# Configure Iterative Refinement
refiner = Refiner(
    maxiter=maxiter : int,
    tol=tol : float,
)

# Configure the backend
backend = BackendConfig(
    provider=provider : str, # ibm, quantinuum, etc.
    instance=instance : Instance(),
    shots=shots : int,
)

In [ ]:
# Configure the solver
solver = Solver(
    algorithm=algorithm : Algorithm(),
    backend=backend : BackendConfig(),
    refiner=refiner : Refiner(), # or None
)

## Execution and Postprocessing

In [ ]:
job = solver.solve(A=A : np.ndarray, b=b : np.ndarray, verbose=True)

job_id = job.job_id
print(f"Submitted to {backend} with job ID: {job_id}")

while True:
    print(f"Waiting for job {job_id} to complete... (status: {job.status()})",  end='\r', flush=True)
    if job.status() in ['DONE', 'CANCELED', 'ERROR']:
        print(f"Job {job_id} completed with status: {job.status()}")
        break
    time.sleep(30)

print(f"\nJob {job_id} finished!")

# Retrieve the optimization results
result = job.result()
print("Solver complete!")

In [ ]:
solution = result['x']
solution_info = result['info']

counts = solution_info['counts']
residual_list = solution_info['residuals']
error_list = solution_info['errors']

In [ ]:
result.plot_residuals()
result.plot_errors()